# Introdução ao LangSmith

O LangSmith é uma plataforma para testar, depurar e avaliar aplicativos baseados em LLM.
Neste notebook, vamos explorar os conceitos fundamentais:
1. **Tracing**: Como visualizar o fluxo de execução das suas chains.
2. **Datasets**: Como criar conjuntos de dados para testes.
3. **Avaliação**: Como rodar testes automatizados nos seus datasets.


# Explicação Detalhada do Assunto

# Introdução ao LangSmith

Bem-vindo ao fascinante mundo do LangSmith, a plataforma essencial para transformar seus aplicativos baseados em LLM de protótipos promissores em soluções robustas e confiáveis. Neste notebook, embarcaremos em uma jornada prática para desvendar os segredos do LangSmith e capacitá-lo a dominar o teste, a depuração e a avaliação de seus projetos de IA Generativa.

## Resumo Executivo

Este notebook serve como um guia introdutório ao LangSmith, explorando seus conceitos fundamentais e demonstrando como utilizá-lo para aprimorar a qualidade e a confiabilidade de seus aplicativos de linguagem. Através de exemplos práticos, você aprenderá a rastrear execuções, criar datasets para testes e avaliar o desempenho de seus modelos.

## Conceitos Chave

Antes de mergulharmos no código, é importante compreender alguns conceitos-chave que permeiam o LangSmith e o ecossistema LangChain:

*   **Tracing:** O tracing permite visualizar o fluxo de execução do seu aplicativo LangChain, desde a entrada até a saída final, revelando insights valiosos sobre o comportamento do seu modelo e identificando gargalos de desempenho. Pense nisso como um raio-x do seu aplicativo!
*   **Datasets:** Datasets são coleções de exemplos cuidadosamente selecionados que representam os diferentes cenários que seu aplicativo precisa lidar. Eles são cruciais para testar a robustez e a precisão do seu modelo em uma variedade de situações.
*   **Avaliação:** A avaliação é o processo de medir o desempenho do seu modelo em relação a um dataset, fornecendo métricas quantitativas e insights qualitativos que ajudam a identificar áreas de melhoria.

## Objetivos de Aprendizado

Ao concluir este notebook, você será capaz de:

*   Configurar o LangSmith e ativar o tracing em seus aplicativos LangChain.
*   Navegar na interface do LangSmith e interpretar os traces de execução.
*   Criar datasets personalizados para testar e avaliar seus modelos.
*   Executar avaliações automatizadas e analisar os resultados no painel do LangSmith.
*   Compreender o papel do LangSmith no ciclo de vida de desenvolvimento de aplicativos de IA Generativa.

## Importância no Ecossistema LangChain

O LangSmith é uma peça fundamental do ecossistema LangChain, preenchendo uma lacuna crítica no desenvolvimento de aplicativos baseados em LLM. Ao fornecer ferramentas poderosas para teste, depuração e avaliação, o LangSmith permite que você:

*   **Reduza o tempo de desenvolvimento:** Identifique e corrija erros mais rapidamente, acelerando o ciclo de desenvolvimento.
*   **Melhore a qualidade do seu aplicativo:** Garanta que seu modelo funcione de forma confiável e precisa em uma variedade de cenários.
*   **Aumente a confiança:** Valide o desempenho do seu modelo e tome decisões informadas sobre como otimizá-lo.

Prepare-se para dominar o LangSmith e elevar seus projetos de IA Generativa a um novo patamar! Vamos começar explorando o tracing básico.

---


In [1]:
### INJECTION START ###
import os
from dotenv import load_dotenv
import sys
for p in ['.', '..', 'scripts', '../scripts']:
    path = os.path.join(p, '.env')
    if os.path.exists(path):
        load_dotenv(path)
        break
if os.getenv('GOOGLE_API_KEY'):
    os.environ['GOOGLE_API_KEY'] = os.getenv('GOOGLE_API_KEY')
    os.environ['GOOGLE_API_KEY'] = os.getenv('GOOGLE_API_KEY')
### INJECTION END ###

import os
from dotenv import load_dotenv
load_dotenv()

%pip install -qU langsmith langchain langchain-openai

You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import getpass

# LangSmith Config
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"

if not os.environ.get("LANGCHAIN_API_KEY"):
    os.environ["LANGCHAIN_API_KEY"] = os.getenv("GOOGLE_API_KEY")

if not os.environ.get("LANGCHAIN_PROJECT"):
    os.environ["LANGCHAIN_PROJECT"] = "curso-langsmith-intro" # Nome do projeto no LangSmith

# GoogleGenerativeAI API Key
if not os.environ.get("GOOGLE_API_KEY"):
    os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")

## 1. Tracing Básico

O tracing já está ativo porque definimos `LANGCHAIN_TRACING_V2="true"`. Qualquer chamada feita com componentes do LangChain será registrada automaticamente.

In [3]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash")

# Simples chamada, isso vai gerar um trace
response = llm.invoke("Explique o que é LangSmith em uma frase.")
print(response.content)

/Users/naubergois/Library/Python/3.9/lib/python/site-packages/google/api_core/_python_version_support.py:246: FutureWarning: You are using a non-supported Python version (3.9.6). Google will not post any further updates to google.api_core supporting this Python version. Please upgrade to the latest Python version, or at least Python 3.10, and then update google.api_core.
  warnings.warn(message, FutureWarning)
/Users/naubergois/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/naubergois/Library/Python/3.9/lib/python/site-packages/google/auth/__init__.py:54: FutureWarning: You are using a Python version 3.9 past its end of life. Google will update google-auth with critical bug fixes on a best-effort basis, but not with any other fixes or features. Please upgrade your Python

/Users/naubergois/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/naubergois/Library/Python/3.9/lib/python/site-packages/langchain_google_genai/chat_models.py:47: FutureWarning: 

All support for the `google.generativeai` package has ended. It will no longer be receiving 
updates or bug fixes. Please switch to the `google.genai` package as soon as possible.
See README for more details:

https://github.com/google-gemini/deprecated-generative-ai-python/blob/main/README.md

  from google.generativeai.caching import CachedContent  # type: ignore[import]


Failed to multipart ingest runs: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


LangSmith é uma plataforma de desenvolvedor para depurar, testar, avaliar e monitorar aplicativos construídos com LLMs (Large Language Models) e cadeias LangChain.


Vá para [smith.langchain.com](https://smith.langchain.com) e verifique o projeto `curso-langsmith-intro`. Você deve ver o trace da execução acima.

## 2. Datasets

Datasets são coleções de exemplos (entradas e saídas esperadas) usados para testar e avaliar seu aplicativo.

In [4]:
from langsmith import Client

client = Client()

dataset_name = "Perguntas de Exemplo"

# Criar Dataset se não existir
if not client.has_dataset(dataset_name=dataset_name):
    dataset = client.create_dataset(
        dataset_name=dataset_name,
        description="Um dataset de exemplo com perguntas e respostas."
    )
    
    # Adicionar exemplos
    client.create_examples(
        inputs=[
            {"pergunta": "Qual é a capital da França?"},
            {"pergunta": "Quanto é 2 + 2?"},
            {"pergunta": "Quem escreveu Dom Casmurro?"}
        ],
        outputs=[
            {"resposta": "Paris"},
            {"resposta": "4"},
            {"resposta": "Machado de Assis"}
        ],
        dataset_id=dataset.id
    )
    print("Dataset criado!")
else:
    print("Dataset já existe.")

LangSmithError: Failed to GET /datasets in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/datasets?limit=1&name=Perguntas+de+Exemplo', '{"detail":"Forbidden"}')

## 3. Avaliação

Agora vamos usar o dataset criado para avaliar nosso modelo.

In [5]:
from langchain.smith import RunEvalConfig, run_on_dataset

# 1. Definir o que vamos testar (nosso "sistema")
# O sistema deve aceitar a entrada do dataset e produzir uma saída.

def meu_app(inputs):
    return llm.invoke(inputs["pergunta"]).content

# 2. Configurar Avaliadores
# Avaliadores "QA" verificam a precisão comparando com a resposta de referência (output do dataset)
eval_config = RunEvalConfig(
    evaluators=["qa"], # usa um LLM para julgar se a resposta bate com o gabarito
)

# 3. Rodar Avaliação
results = run_on_dataset(
    client=client,
    dataset_name=dataset_name,
    llm_or_chain_factory=meu_app,
    evaluation=eval_config,
)

print(results)

LangSmithError: Failed to GET /datasets in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/datasets?limit=1&name=Perguntas+de+Exemplo', '{"detail":"Forbidden"}')

Agora você pode visualizar os resultados da avaliação no painel do LangSmith.